In [95]:
import numpy as np
import pandas as pd 

In [96]:
dados = pd.read_csv("dadoss.csv", decimal= ",", delimiter =";")
dados = dados[["SAPR4", "ELET6", "VIVT3","MGLU3"]]
retornos = dados.pct_change().dropna()
cov = np.array(retornos.cov())
e_r = np.array(retornos.mean()).reshape(4, 1)

In [97]:
def generate_J(e_r, sigma, w, v):
    n = e_r.shape[0]
    e = np.ones(n).reshape((n,1))
    
    a = np.hstack([2*sigma, -1*e_r, -1*e, -1*np.ones(n**2).reshape((n, n))])
    b = np.hstack([np.vstack([e_r.T,e.T ]), np.zeros(2*(n+2)).reshape(2, n+2)])
    c = np.hstack([np.diagflat(v), np.zeros(2*n).reshape(n,2), np.diagflat(w)])
    
    return np.vstack([a, b, c])

In [98]:
def generate_F(e_r, sigma, w, l, v, rp):
    n = e_r.shape[0]
    e = np.ones(n).reshape((n,1))
    
    a = sigma@w - l[0]*e_r - l[1]*e - (np.ones(n)*np.sum(v)).reshape((n,1))
    b = w.T@e_r-rp
    c = w.T@e-1
    d = np.multiply(w, v)
    
    return np.vstack([a,b,c,d])

In [112]:
w = np.array([[0.1],[0], [0.9], [0]])
l = np.random.randn(2).reshape((2,1))
v = np.ones(4).reshape((4,1))
n=4

In [113]:
for i in range(1000):
    J = generate_J(e_r, cov, w, v)
    F = generate_F(e_r, cov, w, l, v,0.00045098 )
    x = -1*np.linalg.inv(J)@F
    w = w + x[0:n][:]
    l = l + x[n:n+2][:]
    v = v + x[n+2:][:]

LinAlgError: Singular matrix

In [111]:
w.T@cov@w

cov

array([[0.00055249, 0.0003446 , 0.00014209, 0.00028484],
       [0.0003446 , 0.00124841, 0.00017635, 0.00045384],
       [0.00014209, 0.00017635, 0.00035113, 0.00020875],
       [0.00028484, 0.00045384, 0.00020875, 0.00133714]])

In [70]:
def minimize_vol(er, cov, rp):
    n = e_r.shape[0]
    w = np.ones(n).reshape((n,1))*1/n
    l = np.zeros(2).reshape((2,1))
    v = np.zeros(n).reshape((n,1))
    
    for i in range(1000):
        J = generate_J(e_r, cov, w, v)
        F = generate_F(e_r, cov, w, l, v,0.00045098 )
        x = -1*np.linalg.inv(J)@F
        w = x[0:n][:]
        l = x[n:n+2][:]
        v = x[n+2:][:]